# Nearest Neighbor Constructive Heuristic
**Author: Dr Bruce Cox  <br/>
Last Revsision: 19 Oct 2023**

This code takes in any arbitrary sized symmetric matrix of distances for the Traveling Salesman Problem and constructs valid tours for each starting city using the node-greedy nearest neighbor heursitic.  

# Import Libraries

In [1]:
import pandas as pd

#### Read in a TSP Instance

In [2]:
tsp_data = pd.read_excel("Lab Data.xlsx", index_col=None)
print(tsp_data.shape)
print(tsp_data)

(15, 15)
    0    1   2   3   4   5    6   7    8   9   10  11  12  13   14
0    0   91  15  99  26  93   69  20   98  62  90  47  23  77    4
1   91    0  50  20  40  59  100  59   13  84  96  68  54  11    1
2   15   50   0  76  96  40    4  53   93  62  59  95  23  97   53
3   99   20  76   0  75  73    5  80   57  12  65  32  94  76   24
4   26   40  96  75   0  65   28  45   12  82  85   3  49  47   80
5   93   59  40  73  65   0    9  81   51  28  93  90  60  73   88
6   69  100   4   5  28   9    0   6   65  12  35   8  61  89   44
7   20   59  53  80  45  81    6   0   77  34   2  36  49  67   99
8   98   13  93  57  12  51   65  77    0  27  62  68  28  38  100
9   62   84  62  12  82  28   12  34   27   0   7  28  49   8    9
10  90   96  59  65  85  93   35   2   62   7   0  97  95  98   26
11  47   68  95  32   3  90    8  36   68  28  97   0  74  34   89
12  23   54  23  94  49  60   61  49   28  49  95  74   0  55   78
13  77   11  97  76  47  73   89  67   38   8  98  34

#### Find the set of valid neighbors for the current city
Since the TSP is fully connected every unvisited city is a valid neighbor

**Test** <br>
First lets play around with the idea of finding valid neighbors.  We want to create a function that takes in a tour and outputs the subset of nodes which are not in the tour.  Try playing around with the set of nodes in the array test.  
<br>
valid_nbrs = [element for element in range(len(tsp_data)) if element not in test] is a list comprehension.  <br>
Recall they have the form:
newlist = [expression for item in iterable if condition] <br>
See: https://www.w3schools.com/python/python_lists_comprehension.asp for more

In [3]:
test = [0, 5, 6]
#create an element in list valid_nbrs if that element isn't in the list test, do this for every element in tsp_data
valid_nbrs = [element for element in range(len(tsp_data)) if element not in test]
print("Valid neighbors : \n" ,*valid_nbrs)

Valid neighbors : 
 1 2 3 4 7 8 9 10 11 12 13 14


In [4]:
def valid(tour):
    valid_nbrs = [element for element in range(len(tsp_data)) if element not in tour]
    #print("Valid neighbors : \n" ,valid_nbrs)
    return valid_nbrs

#### Find the distance to the nearest valid neighbor

In [5]:
# Given current city find nearest valid neighbor
def nearest(current_city):
    smallest = 100**10  #sets an arbitarly large value as our starting 'shortest arc'
    min_position = 0 
    
    for row in tsp_data.itertuples():  #for every row (equiv. every city) in my instance
        if row[0] in valid(tour):  #if that city is not in tour, i.e. it is in valid_nbr
            if(smallest > row[current_city + 1]):  #if the distance to that city is shorter then smallest
                smallest = row[current_city + 1]   #then we choose that arc
                min_position = row[0]              #we need current_city + 1 due to Pythons 0 index, so city 5 is the 6th column
    
    return min_position, smallest

#### Find a nearest neighbor tour given a starting city 

In [6]:
def NN(starting_city, tour):   
    tour_length = 0
    current_city = starting_city
    tour.append(current_city)   #add starting city to tour    
    
    for i in range(len(tsp_data)-1):
        min_position, smallest = nearest(current_city) #make call to function "nearest" to get nearest nbr with arc cost
        tour.append(min_position)
        current_city = min_position
        tour_length += smallest
    
    #The TSP requires that the salesman goes back home at the end.
    tour.append(starting_city)
    tour_length += tsp_data[tour[len(tsp_data)-1]][starting_city]
    
    return tour, tour_length

In [7]:
tour = []
tour_length = 0

In [8]:
NN(1,tour)  #This finds the nearest neighbor tour starting at city 1.

([1, 14, 0, 2, 6, 3, 9, 10, 7, 11, 4, 8, 12, 13, 5, 1], 316)

### Find all the Nearest Neighbor Tours

In [9]:
for i in range(len(tsp_data)):
    tour = []
    tour_length = 0
    tour, tour_length = NN(i, tour)
    print(tour, tour_length)

[0, 14, 1, 13, 9, 10, 7, 6, 2, 12, 8, 4, 11, 3, 5, 0] 307
[1, 14, 0, 2, 6, 3, 9, 10, 7, 11, 4, 8, 12, 13, 5, 1] 316
[2, 6, 3, 9, 10, 7, 0, 14, 1, 13, 11, 4, 8, 12, 5, 2] 243
[3, 6, 2, 0, 14, 1, 13, 9, 10, 7, 11, 4, 8, 12, 5, 3] 269
[4, 11, 6, 2, 0, 14, 1, 13, 9, 10, 7, 12, 8, 5, 3, 4] 339
[5, 6, 2, 0, 14, 1, 13, 9, 10, 7, 11, 4, 8, 12, 3, 5] 307
[6, 2, 0, 14, 1, 13, 9, 10, 7, 11, 4, 8, 12, 5, 3, 6] 269
[7, 10, 9, 13, 1, 14, 0, 2, 6, 3, 11, 4, 8, 12, 5, 7] 273
[8, 4, 11, 6, 2, 0, 14, 1, 13, 9, 10, 7, 12, 5, 3, 8] 314
[9, 10, 7, 6, 2, 0, 14, 1, 13, 11, 4, 8, 12, 5, 3, 9] 272
[10, 7, 6, 2, 0, 14, 1, 13, 9, 3, 11, 4, 8, 12, 5, 10] 291
[11, 4, 8, 1, 14, 0, 2, 6, 3, 9, 10, 7, 12, 13, 5, 11] 345
[12, 0, 14, 1, 13, 9, 10, 7, 6, 2, 5, 8, 4, 11, 3, 12] 298
[13, 9, 10, 7, 6, 2, 0, 14, 1, 8, 4, 11, 3, 5, 12, 13] 295
[14, 1, 13, 9, 10, 7, 6, 2, 0, 12, 8, 4, 11, 3, 5, 14] 313


In [10]:
type(tour)

list